In [14]:
# %% ----------------------------- DEPENDENCIES -------------------------------
import os
import warnings
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject



In [15]:
# Helper: in-memory resample of a single-band array from src_ds to ref_ds grid
def _resample_cfl_to_bp_grid(src_ds, ref_ds, resampling=Resampling.nearest, dst_nodata=np.nan):
    out = np.full((ref_ds.height, ref_ds.width), dst_nodata, dtype=src_ds.dtypes[0])
    reproject(
        source=rasterio.band(src_ds, 1),
        destination=out,
        src_transform=src_ds.transform,
        src_crs=src_ds.crs,
        src_nodata=src_ds.nodata if src_ds.nodata is not None else np.nan,
        dst_transform=ref_ds.transform,
        dst_crs=ref_ds.crs,
        dst_nodata=dst_nodata,
        resampling=resampling,
    )
    # Return a masked array consistent with downstream logic
    mask = np.zeros_like(out, dtype=bool)
    if np.isnan(dst_nodata) and np.issubdtype(out.dtype, np.floating):
        mask = ~np.isfinite(out)
    else:
        mask = (out == dst_nodata)
    return np.ma.array(out, mask=mask)

In [2]:


# %% --------------------------- IFTDSS MATRIX & BINS -------------------------
# IFTDSS Integrated Hazard lookup (rows=CFL bin, cols=BP % of max bin)
# CFL bins: 0:0–2, 1:2–4, 2:4–6, 3:6–8, 4:8–12, 5:>12 (ft)
# BP  bins: 0:0–20%, 1:20–40%, 2:40–60%, 3:60–80%, 4:80–100%
IH_LOOKUP = np.array([
    [1, 1, 1, 1, 2],  # CFL 0–2
    [1, 1, 2, 2, 3],  # 2–4
    [1, 2, 3, 3, 4],  # 4–6
    [2, 3, 3, 4, 4],  # 6–8
    [3, 3, 4, 4, 5],  # 8–12
    [4, 4, 4, 5, 5],  # >12
], dtype=np.int8)

def bin_bp_pct(bp_pct: np.ndarray) -> np.ndarray:
    """Map BP_pct ∈ [0,1] to 5 ordinal classes (0..4) with half-open bins."""
    out = np.full(bp_pct.shape, 4, dtype=np.uint8)  # default top
    out[bp_pct < 0.80] = 3
    out[bp_pct < 0.60] = 2
    out[bp_pct < 0.40] = 1
    out[bp_pct < 0.20] = 0
    return out

def bin_cfl_ft(cfl: np.ndarray) -> np.ndarray:
    """Map CFL (ft) to 6 ordinal classes (0..5): 0–2, 2–4, 4–6, 6–8, 8–12, >12."""
    out = np.full(cfl.shape, 5, dtype=np.uint8)   # default >12
    m = cfl < 12; out[m] = 4
    m = cfl < 8;  out[m] = 3
    m = cfl < 6;  out[m] = 2
    m = cfl < 4;  out[m] = 1
    m = cfl < 2;  out[m] = 0
    return out

def write_like(ref_ds, out_path, arr, dtype=None, nodata=None, compress="lzw"):
    prof = ref_ds.profile.copy()
    prof.update({
        "count": 1,
        "dtype": (dtype or arr.dtype),
        "compress": compress,
        "tiled": True,
        "interleave": "band",
    })
    if nodata is not None:
        prof["nodata"] = nodata
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    data = arr
    if np.ma.isMaskedArray(arr):
        data = np.array(arr.filled(nodata if nodata is not None else np.nan))
    with rasterio.open(out_path, "w", **prof) as dst:
        dst.write(data, 1)



In [ ]:
# # %% ------------------------- COMPUTE METRICS --------------------------
# def compute_ih_from_rasters(bp_path: str,
#                             cfl_path: str,
#                             output_dir: str,
#                             out_names: dict | None = None) -> dict:
#     """
#     Runs the IH workflow for a single fireshed (aligned, clipped rasters).
#     Returns a small summary dict with paths and stats.
#     """
#     os.makedirs(output_dir, exist_ok=True)

#     names = {
#         "IH":       "IH_class.tif",
#         "HAZ":      "hazard_bp_x_cfl.tif",
#         "BP_PCT":   "BP_pct_of_max.tif",
#         "BP_BIN":   "BP_bin.tif",
#         "CFL_BIN":  "CFL_bin.tif",
#     }
#     if out_names:
#         names.update(out_names)

#     # Open rasters
#     with rasterio.open(bp_path) as bp_ds, rasterio.open(cfl_path) as cfl_ds:
#         # --- CRS check ------------------------------------------------------------
#         if bp_ds.crs != cfl_ds.crs:
#             print("\n❌ CRS mismatch detected:")
#             print(f"  BP : {bp_path}")
#             print(f"    → {bp_ds.crs}")
#             print(f"  CFL: {cfl_path}")
#             print(f"    → {cfl_ds.crs}")
#             raise ValueError("CRS mismatch between BP and CFL rasters.")

#         # --- Grid / transform / size check ----------------------------------------
#         grid_ok = (
#             (bp_ds.transform == cfl_ds.transform)
#             and (bp_ds.width == cfl_ds.width)
#             and (bp_ds.height == cfl_ds.height)
#         )
#         if not grid_ok:
#             print("\n❌ Grid alignment mismatch detected:")
#             print(f"  BP  transform : {bp_ds.transform}")
#             print(f"  CFL transform : {cfl_ds.transform}")
#             print(f"  BP  size (width × height): {bp_ds.width} × {bp_ds.height}")
#             print(f"  CFL size (width × height): {cfl_ds.width} × {cfl_ds.height}")
#             raise ValueError("Grid alignment mismatch (transform/size) between BP and CFL rasters.")

#         bp  = bp_ds.read(1, masked=True)
#         cfl = cfl_ds.read(1, masked=True)

#         # Valid where both rasters have data
#         valid = (~bp.mask) & (~cfl.mask)

#         # BP percent-of-max (within this clipped raster)
#         bp_max = float(bp.data[valid].max()) if valid.any() else 0.0
#         bp_pct = np.ma.zeros_like(bp, dtype="float32"); bp_pct.mask = ~valid
#         bp_pct[valid] = (bp.data[valid] / bp_max) if bp_max > 0 else 0.0

#         # Bins
#         bp_bin  = np.ma.zeros_like(bp,  dtype="uint8");  bp_bin.mask  = ~valid
#         cfl_bin = np.ma.zeros_like(cfl, dtype="uint8");  cfl_bin.mask = ~valid
#         bp_bin.data[valid]  = bin_bp_pct(bp_pct.data[valid])
#         cfl_bin.data[valid] = bin_cfl_ft(cfl.data[valid])

#         # Continuous hazard (optional): BP * CFL
#         hazard = np.ma.zeros_like(bp, dtype="float32"); hazard.mask = ~valid
#         hazard[valid] = bp.data[valid] * cfl.data[valid]

#         # Integrated Hazard via lookup (1..5), then PBurn==0 -> 0
#         IH = np.ma.zeros_like(bp, dtype="int16"); IH.mask = ~valid
#         rows = cfl_bin.data[valid].astype(np.intp)
#         cols = bp_bin.data[valid].astype(np.intp)
#         IH_vals = IH_LOOKUP[rows, cols].astype(np.int16)
#         IH.data[valid] = IH_vals

#         pburn_zero = valid & (bp.data == 0)
#         IH.data[pburn_zero] = 0

#         # Write outputs
#         write_like(bp_ds, os.path.join(output_dir, names["BP_PCT"]),  bp_pct,  dtype="float32", nodata=np.nan)
#         write_like(bp_ds, os.path.join(output_dir, names["BP_BIN"]),  bp_bin,  dtype="uint8",  nodata=255)
#         write_like(bp_ds, os.path.join(output_dir, names["CFL_BIN"]), cfl_bin, dtype="uint8",  nodata=255)
#         write_like(bp_ds, os.path.join(output_dir, names["HAZ"]),     hazard,  dtype="float32", nodata=np.nan)
#         write_like(bp_ds, os.path.join(output_dir, names["IH"]),      IH,      dtype="int16",   nodata=-9999)

#     return {
#         "bp_path": bp_path,
#         "cfl_path": cfl_path,
#         "output_dir": output_dir,
#         "bp_max_within_clip": bp_max,
#         "pct_valid": float(valid.sum()) / float(valid.size) if valid.size else 0.0,
#         "written": {k: os.path.join(output_dir, v) for k, v in names.items()},
#     }

In [ ]:
def compute_ih_from_rasters(bp_path: str,
                            cfl_path: str,
                            output_dir: str,
                            out_names: dict | None = None,
                            on_mismatch: str = "error",          # "error" | "warn_proceed" | "auto_resample"
                            resampling: Resampling = Resampling.nearest,
                            dst_nodata_float: float = np.nan,
                            dst_nodata_uint: int = 255,
                            ih_nodata_int: int = -9999) -> dict:
    """
    Runs the IH workflow for a single fireshed.

    Parameters
    ----------
    on_mismatch:
        - "error": raise on grid mismatch (transform/size)
        - "warn_proceed": proceed without fixing (use as-is)
        - "auto_resample": resample CFL in-memory to BP grid (nearest)

    resampling:
        Resampling method if on_mismatch == "auto_resample". Default NEAREST.

    Returns a summary dict (paths, stats, and flags).
    """
    os.makedirs(output_dir, exist_ok=True)

    names = {
        "IH":       "IH_class.tif",
        "HAZ":      "hazard_bp_x_cfl.tif",
        "BP_PCT":   "BP_pct_of_max.tif",
        "BP_BIN":   "BP_bin.tif",
        "CFL_BIN":  "CFL_bin.tif",
    }
    if out_names:
        names.update(out_names)

    with rasterio.open(bp_path) as bp_ds, rasterio.open(cfl_path) as cfl_ds:
        # --- CRS check ------------------------------------------------------------
        if bp_ds.crs != cfl_ds.crs:
            print("\n❌ CRS mismatch detected:")
            print(f"  BP : {bp_path}\n    → {bp_ds.crs}")
            print(f"  CFL: {cfl_path}\n    → {cfl_ds.crs}")
            raise ValueError("CRS mismatch between BP and CFL rasters.")

        # --- Grid check -----------------------------------------------------------
        grids_match = (
            (bp_ds.transform == cfl_ds.transform) and
            (bp_ds.width == cfl_ds.width) and
            (bp_ds.height == cfl_ds.height)
        )

        # Read BP now (masked)
        bp = bp_ds.read(1, masked=True)

        resampled = False
        proceeded_misaligned = False

        if grids_match:
            # Safe to read CFL directly
            cfl = cfl_ds.read(1, masked=True)
        else:
            print("\n❌ Grid alignment mismatch detected:")
            print(f"  BP  transform : {bp_ds.transform}")
            print(f"  CFL transform : {cfl_ds.transform}")
            print(f"  BP  size (width × height): {bp_ds.width} × {bp_ds.height}")
            print(f"  CFL size (width × height): {cfl_ds.width} × {cfl_ds.height}")

            if on_mismatch == "error":
                raise ValueError("Grid alignment mismatch (transform/size) between BP and CFL rasters.")

            elif on_mismatch == "warn_proceed":
                warnings.warn("Proceeding with MISALIGNED rasters (no resample). Results near edges may be unreliable.")
                cfl = cfl_ds.read(1, masked=True)
                proceeded_misaligned = True

            elif on_mismatch == "auto_resample":
                warnings.warn("Auto-resampling CFL to BP grid (nearest) before compute.")
                # Choose a float nodata for resample target to preserve CFL range
                dst_nodata = dst_nodata_float if np.issubdtype(np.dtype(cfl_ds.dtypes[0]), np.floating) else None
                cfl = _resample_cfl_to_bp_grid(cfl_ds, bp_ds, resampling=resampling,
                                               dst_nodata=dst_nodata_float)
                resampled = True

            else:
                raise ValueError(f"Invalid on_mismatch option: {on_mismatch}")

        # Valid where both rasters have data
        valid = (~bp.mask) & (~cfl.mask)

        # BP percent-of-max (within this clipped raster or ref extent)
        bp_max = float(bp.data[valid].max()) if valid.any() else 0.0
        bp_pct = np.ma.zeros_like(bp, dtype="float32"); bp_pct.mask = ~valid
        bp_pct[valid] = (bp.data[valid] / bp_max) if bp_max > 0 else 0.0

        # Bins
        bp_bin  = np.ma.zeros_like(bp,  dtype="uint8");  bp_bin.mask  = ~valid
        cfl_bin = np.ma.zeros_like(cfl, dtype="uint8");  cfl_bin.mask = ~valid
        bp_bin.data[valid]  = bin_bp_pct(bp_pct.data[valid])
        cfl_bin.data[valid] = bin_cfl_ft(cfl.data[valid])

        # Continuous hazard: BP * CFL
        hazard = np.ma.zeros_like(bp, dtype="float32"); hazard.mask = ~valid
        hazard[valid] = bp.data[valid] * cfl.data[valid]

        # Integrated Hazard via lookup (1..5), then PBurn==0 -> 0
        IH = np.ma.zeros_like(bp, dtype="int16"); IH.mask = ~valid
        rows = cfl_bin.data[valid].astype(np.intp)
        cols = bp_bin.data[valid].astype(np.intp)
        IH_vals = IH_LOOKUP[rows, cols].astype(np.int16)
        IH.data[valid] = IH_vals

        pburn_zero = valid & (bp.data == 0)
        IH.data[pburn_zero] = 0

        # Write outputs (match BP grid)
        write_like(bp_ds, os.path.join(output_dir, names["BP_PCT"]),  bp_pct,  dtype="float32", nodata=dst_nodata_float)
        write_like(bp_ds, os.path.join(output_dir, names["BP_BIN"]),  bp_bin,  dtype="uint8",  nodata=dst_nodata_uint)
        write_like(bp_ds, os.path.join(output_dir, names["CFL_BIN"]), cfl_bin, dtype="uint8",  nodata=dst_nodata_uint)
        write_like(bp_ds, os.path.join(output_dir, names["HAZ"]),     hazard,  dtype="float32", nodata=dst_nodata_float)
        write_like(bp_ds, os.path.join(output_dir, names["IH"]),      IH,      dtype="int16",   nodata=ih_nodata_int)

    return {
        "bp_path": bp_path,
        "cfl_path": cfl_path,
        "output_dir": output_dir,
        "bp_max_within_clip": bp_max,
        "pct_valid": float(valid.sum()) / float(valid.size) if valid.size else 0.0,
        "written": {k: os.path.join(output_dir, v) for k, v in names.items()},
        "flags": {
            "grids_match": grids_match,
            "resampled_cfl_to_bp": resampled,
            "proceeded_misaligned": proceeded_misaligned,
            "resampling_method": str(resampling),
        }
    }

In [7]:
# %% ------------------------------ BATCH WRAPPER -----------------------------
def run_fireshed_batch(jobs: list[dict]) -> list[dict]:
    """
    jobs: list of dicts, each like:
      {
        "bp": r"...\\fireshedA_BP.tif",
        "cfl": r"...\\fireshedA_CFL_ft.tif",
        "out_dir": r"...\\fireshedA\\outputs",
        # optional:
        "names": {"IH": "IH_class.tif", ...}
      }
    Returns a list of per-job summaries.
    """
    summaries: list[dict] = []
    for i, job in enumerate(jobs, start=1):
        bp_path  = job["bp"]
        cfl_path = job["cfl"]
        out_dir  = job["out_dir"]
        names    = job.get("names")

        print(f"[{i}/{len(jobs)}] Processing\n  BP : {bp_path}\n  CFL: {cfl_path}\n  →  {out_dir}")
        try:
            summary = compute_ih_from_rasters(bp_path, cfl_path, out_dir, out_names=names)
            print(f"  ✓ Done. IH written to: {summary['written']['IH']}")
            summaries.append(summary)
        except Exception as e:
            print(f"  ✗ Failed: {e}")
            # Optionally append an error record:
            summaries.append({"bp_path": bp_path, "cfl_path": cfl_path, "output_dir": out_dir, "error": str(e)})
    return summaries

In [26]:
JOBS = [
    {
        "bp": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\SCW_97th_BP_v2.tif" ,
        "cfl":r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CFL_ft_SCW_v2.tif" ,
        "out_dir": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SCW",
        "names": {"IH": "IH_SCW.tif"}  # optional custom filenames
    }]

In [27]:
# Run the batch
summaries = run_fireshed_batch(JOBS)
# 'summaries' contains paths and quick stats per fireshed

[1/1] Processing
  BP : C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\SCW_97th_BP_v2.tif
  CFL: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CFL_ft_SCW_v2.tif
  →  C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SCW
  ✓ Done. IH written to: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SCW\IH_SCW.tif


In [28]:
summaries

[{'bp_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\Clip_v2\\SCW_97th_BP_v2.tif',
  'cfl_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\_CFL_rasters\\Clip_v2\\CFL_ft_SCW_v2.tif',
  'output_dir': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCW',
  'bp_max_within_clip': 0.10275841504335403,
  'pct_valid': 0.3765794976475567,
  'written': {'IH': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCW\\IH_SCW.tif',
   'HAZ': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCW\\hazard_bp_x_cfl.tif',
   'BP_PCT': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCW\\BP_pct_of_max.tif',
   'BP_BIN': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCW\\BP_bin.tif',
   'CFL_BIN': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCW\\CFL_bin.tif'}}]

In [35]:
JOBS = [
    {
        "bp": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\SCE_97th_BP_v2.tif" ,
        "cfl":r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CFL_ft_SCE_v2.tif" ,
        "out_dir": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SCE",
        "names": {"IH": "IH_SCE.tif"}  # optional custom filenames
    }]
# Run the batch
summaries = run_fireshed_batch(JOBS)
# 'summaries' contains paths and quick stats per fireshed
summaries

[1/1] Processing
  BP : C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\SCE_97th_BP_v2.tif
  CFL: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CFL_ft_SCE_v2.tif
  →  C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SCE
  ✓ Done. IH written to: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SCE\IH_SCE.tif


[{'bp_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\Clip_v2\\SCE_97th_BP_v2.tif',
  'cfl_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\_CFL_rasters\\Clip_v2\\CFL_ft_SCE_v2.tif',
  'output_dir': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCE',
  'bp_max_within_clip': 0.2101859301328659,
  'pct_valid': 0.5568473782198246,
  'written': {'IH': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCE\\IH_SCE.tif',
   'HAZ': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCE\\hazard_bp_x_cfl.tif',
   'BP_PCT': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCE\\BP_pct_of_max.tif',
   'BP_BIN': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCE\\BP_bin.tif',
   'CFL_BIN': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SCE\\CFL_bin.tif'}}]

In [36]:
JOBS = [
    {
        "bp": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\Santa_Ynez_97th_BP_v2.tif",
        "cfl":r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CFL_ft_SY_v2.tif" ,
        "out_dir": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SY",
        "names": {"IH": "IH_SY.tif"}  # optional custom filenames
    }]
# Run the batch
summaries = run_fireshed_batch(JOBS)
# 'summaries' contains paths and quick stats per fireshed
summaries

[1/1] Processing
  BP : C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\Santa_Ynez_97th_BP_v2.tif
  CFL: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CFL_ft_SY_v2.tif
  →  C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SY
  ✓ Done. IH written to: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\SY\IH_SY.tif


[{'bp_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\Clip_v2\\Santa_Ynez_97th_BP_v2.tif',
  'cfl_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\_CFL_rasters\\Clip_v2\\CFL_ft_SY_v2.tif',
  'output_dir': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SY',
  'bp_max_within_clip': 0.08552762120962143,
  'pct_valid': 0.5724761514359418,
  'written': {'IH': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SY\\IH_SY.tif',
   'HAZ': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SY\\hazard_bp_x_cfl.tif',
   'BP_PCT': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SY\\BP_pct_of_max.tif',
   'BP_BIN': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SY\\BP_bin.tif',
   'CFL_BIN': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\SY\\CFL_bin.tif'}}]

In [6]:
JOBS = [
    {
        "bp": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\Cuyama_97th_BP_v2.tif",
        "cfl": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CFL_ft_Cuyama_v2.tif",
        "out_dir": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\Cuyama_delete",
        "names": {"IH": "IH_Cuyama.tif"}  # optional custom filenames
    }]
# Run the batch
summaries = run_fireshed_batch(JOBS)
# 'summaries' contains paths and quick stats per fireshed
summaries

[1/1] Processing
  BP : C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\Cuyama_97th_BP_v2.tif
  CFL: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CFL_ft_Cuyama_v2.tif
  →  C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\Cuyama_delete
  ✗ Failed: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\Cuyama_97th_BP_v2.tif: No such file or directory


[{'bp_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\Clip_v2\\Cuyama_97th_BP_v2.tif',
  'cfl_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\_CFL_rasters\\Clip_v2\\CFL_ft_Cuyama_v2.tif',
  'output_dir': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\Cuyama_delete',
  'error': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\Clip_v2\\Cuyama_97th_BP_v2.tif: No such file or directory'}]

In [12]:

JOBS = [
    {
        "bp": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\BP\Central_Coast_97th_BP_v2.tif",
        "cfl": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CLF_ft\CFL_ft_CC_v2.tif",
        "out_dir": r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\CC",
        "names": {"IH": "IH_CC.tif"}  # optional custom filenames
    }]
# Run the batch
summaries = run_fireshed_batch(JOBS)
# 'summaries' contains paths and quick stats per fireshed
summaries

[1/1] Processing
  BP : C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\BP\Central_Coast_97th_BP_v2.tif
  CFL: C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CLF_ft\CFL_ft_CC_v2.tif
  →  C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\CC

❌ Grid alignment mismatch detected:
  BP  transform : | 30.00, 0.00, 164397.00|
| 0.00,-30.00, 3879984.82|
| 0.00, 0.00, 1.00|
  CFL transform : | 30.00, 0.00, 164397.00|
| 0.00,-30.00, 3880000.00|
| 0.00, 0.00, 1.00|
  BP  size (width × height): 1747 × 1934
  CFL size (width × height): 1747 × 1934
  ✗ Failed: Grid alignment mismatch (transform/size) between BP and CFL rasters.


[{'bp_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\Clip_v2\\BP\\Central_Coast_97th_BP_v2.tif',
  'cfl_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\_CFL_rasters\\Clip_v2\\CLF_ft\\CFL_ft_CC_v2.tif',
  'output_dir': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\CC',
  'error': 'Grid alignment mismatch (transform/size) between BP and CFL rasters.'}]

In [24]:
bp = r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\BP\Central_Coast_97th_BP_v2.tif"
cfl = r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CLF_ft\CFL_ft_CC_v2.tif"
out_dir= r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\IH_Output\CC"

In [27]:
compute_ih_from_rasters(bp, cfl, out_dir, on_mismatch="error")



❌ Grid alignment mismatch detected:
  BP  transform : | 30.00, 0.00, 164397.00|
| 0.00,-30.00, 3879984.82|
| 0.00, 0.00, 1.00|
  CFL transform : | 30.00, 0.00, 164397.00|
| 0.00,-30.00, 3880000.00|
| 0.00, 0.00, 1.00|
  BP  size (width × height): 1747 × 1934
  CFL size (width × height): 1747 × 1934


ValueError: Grid alignment mismatch (transform/size) between BP and CFL rasters.

In [25]:
compute_ih_from_rasters(bp, cfl, out_dir, on_mismatch="warn_proceed")



❌ Grid alignment mismatch detected:
  BP  transform : | 30.00, 0.00, 164397.00|
| 0.00,-30.00, 3879984.82|
| 0.00, 0.00, 1.00|
  CFL transform : | 30.00, 0.00, 164397.00|
| 0.00,-30.00, 3880000.00|
| 0.00, 0.00, 1.00|
  BP  size (width × height): 1747 × 1934
  CFL size (width × height): 1747 × 1934


C:\Users\bsf31\AppData\Local\Temp\ipykernel_24428\2179992657.py:72: UserWarning: Proceeding with MISALIGNED rasters (no resample). Results near edges may be unreliable.
  warnings.warn("Proceeding with MISALIGNED rasters (no resample). Results near edges may be unreliable.")


{'bp_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\Clip_v2\\BP\\Central_Coast_97th_BP_v2.tif',
 'cfl_path': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\_CFL_rasters\\Clip_v2\\CLF_ft\\CFL_ft_CC_v2.tif',
 'output_dir': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\CC',
 'bp_max_within_clip': 0.06609469652175903,
 'pct_valid': 0.6302744429954971,
 'written': {'IH': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\CC\\IH_class.tif',
  'HAZ': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\CC\\hazard_bp_x_cfl.tif',
  'BP_PCT': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\CC\\BP_pct_of_max.tif',
  'BP_BIN': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\CC\\BP_bin.tif',
  'CFL_BIN': 'C:\\Users\\bsf31\\Documents\\data\\NL060\\WFM Outputs\\run_97thV2\\IH_Output\\CC\\CFL_bin.tif'},
 'f

In [ ]:
compute_ih_from_rasters(bp_path, cfl_path, out_dir, on_mismatch="auto_resample")
